In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from scipy import stats

In [2]:
car = pd.read_csv('/Northeastern/Fall 2021/ALY6020 Predictive Analytics/Week 2/CarPrice_Assignment.csv')

In [3]:
car

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [4]:
np.asarray(car)

array([[1, 3, 'alfa-romero giulia', ..., 21, 27, 13495.0],
       [2, 3, 'alfa-romero stelvio', ..., 21, 27, 16500.0],
       [3, 1, 'alfa-romero Quadrifoglio', ..., 19, 26, 16500.0],
       ...,
       [203, -1, 'volvo 244dl', ..., 18, 23, 21485.0],
       [204, -1, 'volvo 246', ..., 26, 27, 22470.0],
       [205, -1, 'volvo 264gl', ..., 19, 25, 22625.0]], dtype=object)

In [6]:
car = car.rename(columns={'fueltype': 'Gas', 'aspiration': 'Turbo', 'doornumber': '4Door',
                         'enginelocation': 'enginefront'})

In [7]:
car['Gas'] = car['Gas'].replace(['gas'],'1')
car['Gas'] = car['Gas'].replace(['diesel'],'0')
car['Turbo'] = car['Turbo'].replace(['turbo'],'1')
car['Turbo'] = car['Turbo'].replace(['std'],'0')
car['4Door'] = car['4Door'].replace(['four'],'1')
car['4Door'] = car['4Door'].replace(['two'],'0')
car['enginefront'] = car['enginefront'].replace(['front'],'1')
car['enginefront'] = car['enginefront'].replace(['rear'],'0')

In [8]:
new_car = pd.get_dummies(car, columns = ['carbody', 'drivewheel', 'enginetype', 'cylindernumber', 'fuelsystem'])

In [9]:
new_car

,car_ID,symboling,CarName,Gas,Turbo,4Door,enginefront,wheelbase,carlength,carwidth,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,1,3,alfa-romero giulia,1,0,0,1,88.6,168.8,64.1,...,0,0,0,0,0,0,0,1,0,0
1,2,3,alfa-romero stelvio,1,0,0,1,88.6,168.8,64.1,...,0,0,0,0,0,0,0,1,0,0
2,3,1,alfa-romero Quadrifoglio,1,0,0,1,94.5,171.2,65.5,...,0,0,0,0,0,0,0,1,0,0
3,4,2,audi 100 ls,1,0,1,1,99.8,176.6,66.2,...,0,0,0,0,0,0,0,1,0,0
4,5,2,audi 100ls,1,0,1,1,99.4,176.6,66.4,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),1,0,1,1,109.1,188.8,68.9,...,0,0,0,0,0,0,0,1,0,0
201,202,-1,volvo 144ea,1,1,1,1,109.1,188.8,68.8,...,0,0,0,0,0,0,0,1,0,0
202,203,-1,volvo 244dl,1,0,1,1,109.1,188.8,68.9,...,0,0,0,0,0,0,0,1,0,0
203,204,-1,volvo 246,0,1,1,1,109.1,188.8,68.9,...,0,0,0,0,0,1,0,0,0,0


In [11]:
new_car['Gas'] = new_car['Gas'].astype(int)
new_car['Turbo'] = new_car['Turbo'].astype(int)
new_car['4Door'] = new_car['4Door'].astype(int)
new_car['enginefront'] = new_car['enginefront'].astype(int)

In [12]:
new_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 51 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   car_ID                 205 non-null    int64  
 1   symboling              205 non-null    int64  
 2   CarName                205 non-null    object 
 3   Gas                    205 non-null    int32  
 4   Turbo                  205 non-null    int32  
 5   4Door                  205 non-null    int32  
 6   enginefront            205 non-null    int32  
 7   wheelbase              205 non-null    float64
 8   carlength              205 non-null    float64
 9   carwidth               205 non-null    float64
 10  carheight              205 non-null    float64
 11  curbweight             205 non-null    int64  
 12  enginesize             205 non-null    int64  
 13  boreratio              205 non-null    float64
 14  stroke                 205 non-null    float64
 15  compre

In [15]:
y=new_car[['price']]
x=new_car.iloc[:, 3:19]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [16]:
X2 = sm.add_constant(x_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     57.48
Date:                Sun, 03 Oct 2021   Prob (F-statistic):           6.53e-55
Time:                        00:02:01   Log-Likelihood:                -1538.6
No. Observations:                 164   AIC:                             3111.
Df Residuals:                     147   BIC:                             3164.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -2.526e+04   2.09e+04  